### Exercício: Análise Manual do Dataset Disney+

**Objetivo:**

Este exercício tem como objetivo desenvolver suas habilidades em manipulação manual de dados em Python, processando um dataset de shows da Disney+ sem o auxílio de bibliotecas de parsing como `csv` ou `pandas`. Você deverá extrair informações relevantes e gerar um relatório detalhado.

**Dataset:**

O dataset a ser utilizado está disponível no seguinte link:
[https://www.kaggle.com/datasets/eshummalik/disney](https://www.kaggle.com/datasets/eshummalik/disney)

**Instruções:**

1.  **Download do Dataset:** Baixe o arquivo do dataset manualmente a partir do link fornecido.
2.  **Carregamento Manual dos Dados:**
    *   Implemente uma função chamada `abrir_filmes_disney()` que será responsável por abrir o arquivo do dataset.
    *   Você **não** deve utilizar bibliotecas como `csv` ou `pandas` para ler o arquivo. A leitura deve ser feita linha por linha, utilizando as funcionalidades básicas de manipulação de arquivos em Python.
    *   Ignore linhas vazias encontradas no arquivo.
    *   Para cada linha do dataset, identifique se ela representa um filme (`type` coluna) ou uma série.
    *   Crie duas classes em Python: `Filme` e `Serie`. Cada linha do dataset deve ser mapeada para uma instância da classe `Filme` ou `Serie` correspondente, contendo seus respectivos atributos (baseados nas colunas do dataset). Implemente getters e setters para os atributos conforme necessário.
    *   Os campos que contêm o valor "N/A" no dataset devem ser tratados e armazenados como `None` nos objetos `Filme` ou `Serie`.
    *   A função `abrir_filmes_disney()` deve retornar uma lista contendo todos os objetos `Filme` e `Serie` criados a partir do dataset.
    *   Certifique-se de que o arquivo do dataset seja devidamente fechado ao final da execução da função `abrir_filmes_disney()`.
3.  **Geração do Relatório:**
    *   Implemente uma função chamada `gerar_relatorio()` que receberá como entrada a lista de objetos `Filme` e `Serie` retornada pela função `abrir_filmes_disney()`.
    *   Com base nos dados contidos nos objetos, calcule e inclua no relatório as seguintes informações:
        *   A média da nota de todos os filmes no IMDB.
        *   A média da nota de todos os filmes no Metascore.
        *   A média do número de votos de todos os filmes no IMDB.
        *   As 3 línguas mais usadas e as 3 línguas menos usadas no dataset.
        *   Os 3 atores que mais aparecem e os 3 atores que menos aparecem no dataset.
        *   O diretor com mais filmes no dataset.
        *   O diretor com o filme mais popular no IMDB (considerando a maior nota IMDB).
        *   O diretor com o filme mais popular no Metascore (considerando a maior nota Metascore).
        *   O ano em que mais filmes foram lançados.
        *   A pior série segundo a nota IMDB e a pior série segundo a nota Metascore.
        *   Uma lista de filmes que possuem mais de um lançamento (considerados "remakes" ou diferentes versões no dataset, identificados por títulos iguais mas anos de lançamento diferentes).
    *   O relatório de saída deve ser salvo em um arquivo texto chamado `relatorio-disney.txt` no mesmo diretório do script.
4.  **Estrutura do Código:** Organize seu código de forma modular, utilizando as funções `abrir_filmes_disney()` e `gerar_relatorio()` conforme especificado. Você não é obrigado a passar parâmetros para essas funções, mas pode fazê-lo se julgar necessário para uma melhor organização do código.